In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
# !pip install spektral

In [ ]:
!pip install spektral

In [4]:

#general libraries
import tensorflow as tf
import numpy as np
import os 
import glob 
import networkx as nx
import pdb
import networkx.algorithms.isomorphism as iso
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

#spektral
from spektral.data import Graph
from spektral.data import Dataset
from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.models import GeneralGNN
from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GeneralConv, GatedGraphConv

#tensorflow impors 
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import OGB
from spektral.layers import ECCConv, GlobalSumPool

In [5]:

n1, n2 = 10, 20 # 10 nodes
m1, m2 = 20, 100  # 20 edges
f1 = 100
f2 = 100
seed = 20160  # seed random number generators for reproducibility


In [6]:
from spektral.data import Graph


class MyDataset(Dataset):
    """
    A dataset of random colored graphs.
    The task is to classify each graph with the color which occurs the most in
    its nodes.
    The graphs have `n_colors` colors, of at least `n_min` and at most `n_max`
    nodes connected with probability `p`.
    """

    def __init__(self, n, m, number, **kwargs):
        self.n = n
        self.m = m
        self.number = number
        self.seed = seed 
        self.labels = []
        super().__init__(**kwargs)

    def read(self):
        def make_graph(i):
            G = nx.gnm_random_graph(self.n[i%2], self.m[i%2], seed=seed)
            self.labels.append((i%2))
            return Graph(x=nx.to_numpy_array(G).astype(float), a=csr_matrix(nx.to_numpy_array(G)), y=[float(i%2)])

        # We must return a list of Graph objects
        print(self.number)
        return [make_graph(_) for _ in range(self.number)]


In [7]:

data_train = MyDataset(n=[30,30], m=[38,48], number=500)
data_test = MyDataset(n=[30,30], m=[28,38], number=200)
data_val = MyDataset(n=[30,30], m=[42,52], number=10)

500
200
10


In [8]:


np.shape(data_train.n_labels)

()

In [11]:
print(len(data_train.labels))

500


In [12]:
val_loss_array = []
training_loss = []

In [22]:
from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool

################################################################################
# Config
################################################################################
learning_rate = 1e-3
epochs = 10
batch_size = 8

################################################################################
# Load data
################################################################################
dataset = data_train  # Set amount=None to train on whole dataset

# Parameters
F = dataset.n_node_features  # Dimension of node features
#S = dataset.n_edge_features  # Dimension of edge features


dataset_tr, dataset_val, dataset_te = data_train, data_val, data_test
# dataset_val = data_val

loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_val = DisjointLoader(dataset_val, batch_size=batch_size, epochs=5)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=10)

################################################################################
# Build model
dp_rate = 0.7
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv0 = GCSConv(30, activation="relu")
        self.conv1 = GCSConv(30, activation="relu")
        # self.conv2 = GeneralConv(128, activation="relu", dropout=dp_rate)
        # self.conv3 = GeneralConv(64, activation="relu", dropout=dp_rate)
        self.conv4 = GCSConv(30, activation="relu")
        self.global_pool = GlobalAvgPool()
        self.dense =  Dense(1, activation="softmax")

    def call(self, inputs):
        x, a, i = inputs
        # print(np.shape(a))
        # print(np.shape(i))
        # print(np.shape(x))
        x = self.conv0([x, a])
        x = self.conv1([x, a])
        x = self.conv4([x, a])
        # print(np.shape(x))
        # print(np.shape(a))
        # print(np.shape(i))
        output = self.global_pool([x])
        output = self.dense(output)
        # print(output)
        
        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn1 = CategoricalCrossentropy()
loss_fn2 = CategoricalCrossentropy()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        print("Target: {}".format(target))
        predictions = model(inputs, training=True)
        print("Predictions: {}".format(predictions))
        # print("JFKFKFKFKFK")
        # print(target)
        # print(predictions)
        loss = loss_fn1(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    # print('here')
    step += 1
    loss += train_step(*batch)
    
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        training_loss.append(format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    loss += loss_fn2(target, predictions)
loss /= 100*loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))

Target: Tensor("target:0", shape=(None, 1), dtype=float64)
Predictions: Tensor("net_8/dense_7/Softmax:0", shape=(None, 1), dtype=float32)
Target: Tensor("target:0", shape=(None, 1), dtype=float64)
Predictions: Tensor("net_8/dense_7/Softmax:0", shape=(None, 1), dtype=float32)
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Loss: 0.0
Testing model
Done. Test loss: 0.0


In [15]:
print(predictions)
print(target)

tf.Tensor(
[[0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]], shape=(8, 1), dtype=float32)
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]]


In [126]:
print(loader_tr.dataset)

MyDataset(n_graphs=1000)


In [20]:
################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCSConv(32, activation="relu")
        self.conv2 = GCSConv(32, activation="relu")
        self.conv3 = GCSConv(32, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(1, activation="softmax")

    def call(self, inputs):
        x, a, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        x = self.conv3([x, a])
        output = self.global_pool([x, i])
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(lr=learning_rate)
loss_fn = CategoricalCrossentropy()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc


def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])


epoch = step = 0
best_val_loss = np.inf
best_weights = None
results = []
for batch in loader_tr:
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))
    if step == loader_tr.steps_per_epoch:
        step = 0
        epoch += 1

        # Compute validation loss and accuracy
        val_loss, val_acc = evaluate(loader_val)
        print(
            "Ep. {} - Loss: {:.3f} - Acc: {:.3f} - Val loss: {:.3f} - Val acc: {:.3f}".format(
                epoch, *np.mean(results, 0), val_loss, val_acc
            )
        )

        results = []

################################################################################
# Evaluate model
################################################################################
model.set_weights(best_weights)  # Load best model
test_loss, test_acc = evaluate(loader_te)
print("Done. Test loss: {:.4f}. Test acc: {:.2f}".format(test_loss, test_acc))

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


TypeError: ignored

In [145]:
print(target)

[[0]
 [1]
 [0]
 [1]]


In [147]:
print(predictions)

tf.Tensor([[57.695454]], shape=(1, 1), dtype=float32)


In [151]:
print(inputs[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [129]:
i=0
for batch in loader_te:
  i+=1
print(i)

/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


KeyboardInterrupt: ignored

In [130]:
print(inputs)

(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f0473a27710>, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3]))


In [133]:
np.shape(inputs[0])

(120, 30)

In [173]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.models import GeneralGNN

################################################################################
# Config
################################################################################
batch_size = 1
learning_rate = 0.01
epochs = 4

################################################################################
# Load data
################################################################################
data = data_train

# Train/test split
np.random.shuffle(data)
split = int(0.8 * len(data))
data_tr, data_te = data_train, data_val

# Data loaders
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(data_te, batch_size=batch_size)

################################################################################
# Build model
################################################################################
model = GeneralGNN(data.n_labels, activation="softmax")
optimizer = Adam(learning_rate)
loss_fn = CategoricalCrossentropy()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc


def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
            len(target), 
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])


epoch = step = 0
results = []
for batch in loader_tr:
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))
    if step == loader_tr.steps_per_epoch:
        step = 0
        epoch += 1
        results_te = evaluate(loader_te)
        print(
            "Ep. {} - Loss: {:.3f} - Acc: {:.3f} - Test loss: {:.3f} - Test acc: {:.3f}".format(
                epoch, *np.mean(results, 0), *results_te
            )
        )
        results = []

################################################################################
# Evaluate model
################################################################################
results_te = evaluate(loader_te)
print("Final results - Loss: {:.3f} - Acc: {:.3f}".format(*results_te))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.


Ep. 1 - Loss: 0.000 - Acc: 1.000 - Test loss: 0.000 - Test acc: 1.000


KeyboardInterrupt: ignored

In [ ]:
print(np.shape(data))

In [125]:
inputs

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3]))

In [106]:
print(inputs)

(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 1., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]]), <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f04742d3310>, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3]))


In [ ]:
print(data[0])